# Weekly activity
1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.
2. Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following:  
![bee and flowers](img_embed/activity3.PNG "bee_flower")
3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```

4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [1]:
# setup
import cv2 as cv
import numpy as np

# Sample solution

# Question 1
## First way: using external package: `imutils`
Remember to `pip install imutils`.

In [4]:
!pip install imutils

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25855 sha256=ff886d3f8775f20b9a8bb864d69dc4e09923fcbdc40f371a2ced265eb97f7a2b
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\5b\76\96\ad0c321506837bef578cf3008df3916c23018435a355d9f6b1
Successfully built imutils



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Question 1
#1st way: Utilize external library
from imutils import rotate_bound
img = cv.imread('images/lena.jfif')
rotated = rotate_bound(img, -45)

cv.imshow('rotate', rotated)
cv.waitKey(0)
cv.destroyAllWindows()

## 2nd way: modify the transformation matrix, M

In [4]:
#2nd way: Modify the transformation matrix
# define a custom function
def rotate_image(img, angle, scale):
    """Arguments:
    img: source image
    angle, scale: argument for cv.getRotationMatrix2D
    """
    height, width = img.shape[:2]
    img_center = (int(width/2), int(height/2))
    rotation_mat = cv.getRotationMatrix2D(img_center, angle, scale)
    # result: 2x3 matrix
    abs_cos = abs(rotation_mat[0, 0])
    abs_sin = abs(rotation_mat[1, 0])
    
    new_w = int(abs_sin*height + abs_cos*width)
    new_h = int(abs_cos*height + abs_sin*width)
    
    rotation_mat[0, 2] += new_w/2 - img_center[0]
    rotation_mat[1, 2] += new_h/2 - img_center[1]
    
    rotated_img = cv.warpAffine(img, rotation_mat, (new_w, new_h))
    return rotated_img

rotate_img = rotate_image(img, 45, 1)

cv.imshow('original', img)
cv.imshow('rotate', rotate_img)
cv.waitKey(0)
cv.destroyAllWindows()

## Question 2

In [5]:
img1 = cv.imread('images/native-bee.png')
img2 = cv.imread('images/flower.jfif')

# Add flower on the top left hand corner of the bee image
rows, cols = img2.shape[:2]
roi = img1[:rows, :cols]

# roi is the background; now we need to get the flower
# Lets use image thresholding
img2_gray = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(img2_gray, 70, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

# get the background (from img1) and foreground (flower from img2)
img1_bg = cv.bitwise_and(roi, roi, mask=mask_inv)
img2_fg = cv.bitwise_and(img2, img2, mask=mask)

res = cv.add(img1_bg, img2_fg)
img1[:rows, :cols] = res

cv.imshow("flower_bee", img1)
cv.waitKey(0)
cv.destroyAllWindows()

## Question 3

In [6]:
import random

def random_center_crop(image, min_crop_ratio=0.5, max_crop_ratio=0.9):
    height, width = image.shape[:2]

    # randomly choos crop size
    crop_ratio = random.uniform(min_crop_ratio, max_crop_ratio)
    crop_height = int(height * crop_ratio)
    crop_width = int(width * crop_ratio)

    # calculate crop coordinates
    y1 = (height - crop_height) // 2
    y2 = y1 + crop_height
    x1 = (width - crop_width) // 2
    x2 = x1 + crop_width

    # perform the crop
    cropped_image = image[y1:y2, x1:x2]

    return cropped_image

## Question 4
Another type of common image noise is called salt & pepper noise.

In [9]:
img = cv.imread("images/camera.jpg")

rng = np.random.default_rng(0)
prob_mat = rng.uniform(0, 1, size=img.shape[:2])

degree_noise = 0.05
img_noise = img.copy()

# add noise
img_noise[prob_mat <= degree_noise / 2, ...] = (0, 0, 0)
img_noise[prob_mat >= 1 - (degree_noise / 2), ...] = (255, 255, 255)

cv.imshow("original", img)
cv.imshow("salt & pepper noise", img_noise)
cv.waitKey(0)
cv.destroyAllWindows()